# Fine-tune T5 on CNN/Daily

## Libraries and environment preparation

In [ ]:
#Install essential packages
%%capture
! pip install datasets transformers rouge-score nltk wandb

In [ ]:
#install Git-LFS
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
#Colab Environment Check for GPU and RAM
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

#GPU check
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Your runtime has 13.6 gigabytes of available RAM

Sun Nov 28 00:49:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                      

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [ ]:
# Make sure your version of Transformers is at least 4.11.0 
# to run the following code correctly:
import datasets
import transformers
print(transformers.__version__)

4.12.5


In [ ]:
# Import Wandb 
import os
import wandb
API_KEY = '39991c538626bee25c64d4f8a4c3403dd635537c'
os.environ["WANDB_API_KEY"] = API_KEY

## Loading the dataset

In [ ]:
# import dataset huggingface
raw_datasets = datasets.load_dataset('cnn_dailymail', '3.0.0')

Reusing dataset cnn_dailymail (/root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [ ]:
# Visualize the Data

import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=3):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    del df["id"]
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_datasets["train"])

## Preprocessing the data

In [ ]:
model_checkpoint = "t5-base"

In [ ]:
# Import tokenizer from model checkpoint
from transformers import AutoTokenizer   
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
# If you are using one of the five T5 checkpoints we have to prefix 
# the inputs with "summarize:" (t5 is a multi-task model).

if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

For Xsum, the input tokens are about 1500 and the length of the summaries are about 160. Here we truncate to 1024 and 128

In [ ]:
# tokenlize inputs into map

max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/288 [00:00<?, ?ba/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['article', 'attention_mask', 'highlights', 'id', 'input_ids', 'labels'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'attention_mask', 'highlights', 'id', 'input_ids', 'labels'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'attention_mask', 'highlights', 'id', 'input_ids', 'labels'],
        num_rows: 11490
    })
})

## Fine-tuning the model

In [ ]:
# Import Huggingface Automodel class from model checkpoint

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [ ]:
# data collator: pad the inputs and labels during each batch to save space
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
# keep track with wandb
wandb.init(project="T5-sum")

wandb: Currently logged in as: shusunny (use `wandb login --relogin` to force relogin)


In [ ]:
# Define compute_metrics
import nltk
import numpy as np
nltk.download('punkt')

metric = datasets.load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Define traing args, batch size and epoch

batch_size = 8
epochs = 1
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-cnn",
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    logging_steps=1000,  # set to 1000 for full training
    save_steps=2000,  # set to 500 for full training
    eval_steps=6000,  # set to 8000 for full training
    save_total_limit=3,
    num_train_epochs=epochs,
    predict_with_generate=True,
    fp16=True,
    report_to="wandb",
)

In [ ]:
# Pass into the trainer

train_dataset=tokenized_datasets["train"]
eval_dataset=tokenized_datasets["validation"]

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp fp16 backend


We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: highlights, id, article.
***** Running training *****
  Num examples = 287113
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 35890
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
6000,2.205000,1.920674,23.485500,10.086000,19.024200,21.961600,18.996300
12000,2.218000,1.932958,23.430400,10.068800,18.989800,21.907500,18.995800
18000,2.215300,1.932958,23.430400,10.068800,18.989800,21.907500,18.995800
24000,2.214900,1.932958,23.430400,10.068800,18.989800,21.907500,18.995800


Saving model checkpoint to t5-base-finetuned-cnn/checkpoint-2000
Configuration saved in t5-base-finetuned-cnn/checkpoint-2000/config.json
Model weights saved in t5-base-finetuned-cnn/checkpoint-2000/pytorch_model.bin
tokenizer config file saved in t5-base-finetuned-cnn/checkpoint-2000/tokenizer_config.json
Special tokens file saved in t5-base-finetuned-cnn/checkpoint-2000/special_tokens_map.json
Saving model checkpoint to t5-base-finetuned-cnn/checkpoint-4000
Configuration saved in t5-base-finetuned-cnn/checkpoint-4000/config.json
Model weights saved in t5-base-finetuned-cnn/checkpoint-4000/pytorch_model.bin
tokenizer config file saved in t5-base-finetuned-cnn/checkpoint-4000/tokenizer_config.json
Special tokens file saved in t5-base-finetuned-cnn/checkpoint-4000/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: highlights, id, article.
***** Running Evaluation *****
  

## Trying with a smaller dataset

In [ ]:
# Init new logging params
wandb.init(project="T5-sum")

wandb: Currently logged in as: shusunny (use `wandb login --relogin` to force relogin)


In [ ]:
# Select to get smaller dataset
small_train = raw_datasets['train'].select(list(range(0, 5000)))
small_val = raw_datasets['validation'].select(list(range(0, 500)))
small_train

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 5000
})

In [ ]:
# Processing
tokenized_train = small_train.map(preprocess_function, batched=True)
tokenized_val = small_val.map(preprocess_function, batched=True)
tokenized_train

  0%|          | 0/5 [00:00<?, ?ba/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234/cache-34bfa482b3a753b2.arrow


Dataset({
    features: ['article', 'attention_mask', 'highlights', 'id', 'input_ids', 'labels'],
    num_rows: 5000
})

In [ ]:
# Import a new T5
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model_small = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
# data collator: pad the inputs and labels during each batch to save space
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_small)

In [ ]:
# Define traing args, batch size and epoch
# batch size depends on the memory of GPU

batch_size = 8
epochs = 20
model_name = model_checkpoint.split("/")[-1]
args_small = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-cnn-small",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    save_strategy = "epoch",
    save_total_limit=3,
    num_train_epochs=epochs,
    predict_with_generate=True,
    fp16=True,
    report_to="wandb",
)

In [ ]:
# Pass into the trainer

train_dataset=tokenized_train
eval_dataset=tokenized_val

trainer_small = Seq2SeqTrainer(
    model_small,
    args_small,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp fp16 backend


In [ ]:
trainer_small.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: article, id, highlights.
***** Running training *****
  Num examples = 5000
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 12500
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.719400,1.799034,26.221000,11.039200,21.507700,24.000100,18.986000
2,1.647300,1.794493,26.413400,11.029400,21.615500,24.105100,18.988000
3,1.602500,1.799427,26.467200,11.122700,21.629000,24.071200,18.994000
4,1.549000,1.801260,26.591000,11.122500,21.752300,24.229500,18.984000
5,1.520500,1.788304,26.352800,10.993200,21.671400,24.111500,18.996000
6,1.558500,1.930049,24.152100,9.669600,19.883900,22.165700,19.000000
7,1.777000,2.099338,24.282100,9.340900,19.431300,21.940700,19.000000
8,1.983000,2.083384,24.176300,9.321700,19.468000,21.920200,19.000000
9,1.956100,2.085913,24.388800,9.448400,19.601700,22.059300,19.000000
10,1.951800,2.087633,24.332800,9.384800,19.504300,22.047000,19.000000


The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: article, id, highlights.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to t5-base-finetuned-cnn-small/checkpoint-625
Configuration saved in t5-base-finetuned-cnn-small/checkpoint-625/config.json
Model weights saved in t5-base-finetuned-cnn-small/checkpoint-625/pytorch_model.bin
tokenizer config file saved in t5-base-finetuned-cnn-small/checkpoint-625/tokenizer_config.json
Special tokens file saved in t5-base-finetuned-cnn-small/checkpoint-625/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: article, id, highlights.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to t5-base-finetuned-cnn-small/checkpoint-1250
Configuration saved in t5-bas

TrainOutput(global_step=12500, training_loss=1.8421403125, metrics={'train_runtime': 10104.5209, 'train_samples_per_second': 9.897, 'train_steps_per_second': 1.237, 'total_flos': 7.134444078170112e+16, 'train_loss': 1.8421403125, 'epoch': 20.0})

In [ ]:
wandb.finish()

eval/gen_len,▂▃▅▁▆███████████████
eval/loss,▁▁▁▁▁▄██████████████
eval/rouge1,▇▇██▇▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂
eval/rouge2,████▇▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/rougeL,▇████▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁
eval/rougeLsum,▇████▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▅▃▃▆▄▄▁▄▄▅▄▃█▇▅▄▄█▄▄
eval/samples_per_second,▃▆▆▃▅▅█▅▅▄▅▆▁▂▄▅▅▁▅▅
eval/steps_per_second,▄▆▆▃▅▅█▅▅▄▅▅▁▂▄▄▅▁▅▅
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████


In [ ]:
!ls

sample_data  t5-base-finetuned-cnn  t5-base-finetuned-cnn-small  wandb


In [ ]:
!ls t5-base-finetuned-cnn/

checkpoint-22000  checkpoint-24000  checkpoint-26000


In [ ]:
!ls t5-base-finetuned-cnn-small/

checkpoint-11250  checkpoint-11875  checkpoint-12500


In [ ]:
from transformers import T5Tokenizer, T5Model

model = T5Model.from_pretrained('t5-base-finetuned-cnn/checkpoint-22000')

loading configuration file t5-base-finetuned-cnn/checkpoint-22000/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "prefi

In [ ]:
model_small=T5Model.from_pretrained('t5-base-finetuned-cnn-small/checkpoint-11250')

loading configuration file t5-base-finetuned-cnn-small/checkpoint-11250/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      

In [ ]:
# map data correctly
def generate_summary(batch):
    tokenizer.padding_side = "left"
    tokenizer.pad_token = tokenizer.eos_token # to avoid an error
    # cut off at BERT max length 512
    sentences = [prefix + sentence for sentence in batch["article"]]
    inputs = tokenizer(sentences, truncation=True, max_length=512, return_tensors="pt", padding=True)
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

In [ ]:
test_data = raw_datasets['test']
test_data = test_data.select(range(16))

In [ ]:
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token # to avoid an error

sentences = test_data["article"] 
inputs = tokenizer([prefix + sentence for sentence in sentences], max_length=max_input_length, return_tensors="pt", padding=True)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2229: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  "`max_length` is ignored when `padding`=`True` and there is no truncation strategy. "
